# Seminar 7. Dataclasses

Example from warm up test:

In [ ]:
class SingletonClass:
    _instance = None

    def __new__(cls, *args, **kwargs): 
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance


obj1 = SingletonClass() 
obj2 = SingletonClass() 
assert id(obj1) == id(obj2)
assert obj1 is obj2

Let's create simple decorator. Main points:
- function are objects and we can work with them like with objects before calling them
- "@" syntax just call decorator as a function, passes decorated object as an argument and replaces decorated object with result of this function call
- decorator must be callable object that receives one argument
- decorated object can be any object
- result of decoration can be any object

In [ ]:
def log_call(func):
    # we can return None and foo became None (that is legal, but probably meaningless)
    # return None

    def wrapper(*args, **kwargs):
        print(f'{func.__name__} called with args={args}, kwargs={kwargs}')
        result = func(*args, **kwargs)
        print(f'{func.__name__} result={result}')
        return result

    return wrapper


@log_call         # that is the same as "foo = log_call(foo)"
def foo(a, b=1):
    print(1)
    return 2

print(foo)
print('-' * 10)
print(foo(a=1, b=2))

<function log_call.<locals>.wrapper at 0x7f6a93bca320>
----------
foo called with args=(), kwargs={'a': 1, 'b': 2}
1
foo result=2
2


`wrapper` uses *closure* to achieve correct `func`.

Wrapper created for `bar` function achieves `func` from scope of `log_call` function. When this wrapper was created, scope contained `bar` as a value of `func`. Python keep outer scope (closure) with function. So this wrapper achieves `bar`, not `foo`.

In [ ]:
def log_call(func):

    def wrapper(*args, **kwargs):
        print(f'{func.__name__} called with args={args}, kwargs={kwargs}')
        result = func(*args, **kwargs)
        print(f'{func.__name__} result={result}')
        return result

    return wrapper


@log_call
def foo(a, b=1):
    print(1)
    return 2


@log_call
def bar(a, b=1):
    print(2)
    return 3

print(foo)
print(bar)
print('-' * 10)
print(foo(a=1, b=2))
print('-' * 10)
print(bar(a=1, b=2))

<function log_call.<locals>.wrapper at 0x7f6a93bca8c0>
<function log_call.<locals>.wrapper at 0x7f6a93bcaa70>
----------
foo called with args=(), kwargs={'a': 1, 'b': 2}
1
foo result=2
2
----------
bar called with args=(), kwargs={'a': 1, 'b': 2}
2
bar result=3
3


When we replacing object with other object, we lose name, docstring and other describing properties. That can break some code, depending on them. For example, documentation generator.

In [ ]:
def log_call(func):

    def wrapper(*args, **kwargs):
        print(f'{func.__name__} called with args={args}, kwargs={kwargs}')
        result = func(*args, **kwargs)
        print(f'{func.__name__} result={result}')
        return result

    return wrapper


@log_call
def foo(a, b=1):
    """foo docstring"""
    print(1)
    return 2


print('foo name is', foo.__name__)
print('foo docstring is', foo.__doc__)

foo name is wrapper
foo docstring is None


There is `wraps` decorator from `functools` package to fix it. This is standart practice, like codestyle. Advice you to use it.

In [ ]:
import functools

def log_call(func):

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f'{func.__name__} called with args={args}, kwargs={kwargs}')
        result = func(*args, **kwargs)
        print(f'{func.__name__} result={result}')
        return result

    # We could fix it by ourselves, like this:
    # wrapper.__name__ = func.__name__
    # That is actual example of what `functools.wraps` do.
    # But it is too much job to do... Using a ready decorator is much simplier.

    return wrapper


@log_call
def foo(a, b=1):
    """foo docstring"""
    print(1)
    return 2


print('foo name is', foo.__name__)
print('foo docstring is', foo.__doc__)

foo name is foo
foo docstring is foo docstring


We can write any expression after @. It is only important, that result must be callable object with one argument.

We can use this property and implement the trick to create **parametrized decorator**.

The trick is to wrap our decorator into function that tooks decorator parameters and returns actual decorator. This parameters will be saved in main function scope and available in inner functions.

In the example below, `log_call` keeps the parameters and returns actual decorator `temp`. `temp` decorates the function and replaces it with `wrapper`.

In [ ]:
import functools

def log_call(turn_off_logging=False):

    def temp(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if turn_off_logging is False:
                print(f'{func.__name__} called with args={args}, kwargs={kwargs}')
            result = func(*args, **kwargs)
            if turn_off_logging is False:
                print(f'{func.__name__} result={result}')
            return result

        return wrapper
    
    return temp


@log_call(turn_off_logging=True)
def foo(a, b=1):
    print(1)
    return 2


# that is the same as
# foo = log_call(turn_off_logging=True)(foo)
# or
# temp = log_call(turn_off_logging=True)
# foo = temp(foo)

print(foo)
print('-' * 10)
print(foo(a=1, b=2))

<function foo at 0x7f6a901554d0>
----------
1
2
